In [1]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

In [3]:
const { execSync } = await import('child_process');
console.log(execSync('npm install @viz-js/viz').toString());
import * as fs from "fs";
import { instance } from "@viz-js/viz";
import { nfa2dfa,  NFA, DFA, TransRel  } from "./01-NFA-2-DFA.js";


up to date, audited 13 packages in 1s

1 package is looking for funding
  run `npm fund` for details

found 0 vulnerabilities



Import der TypeScript Datei aus dem letzen Notebook.

In [4]:


function showSet<T>(s: Set<T>): string {
  return `{ ${Array.from(s).join(", ")} }`;
}

const States = new Set(
  Array.from({ length: 8 }, (_, i) => `q${i}`)
);

const Sigma = new Set(["a", "b"]);

const delta: TransRel = new Map([
  ["q0,ε", new Set(["q1", "q2"])],
  ["q1,b", new Set(["q3"])],
  ["q2,a", new Set(["q4"])],
  ["q3,a", new Set(["q5"])],
  ["q4,b", new Set(["q6"])],
  ["q5,ε", new Set(["q7"])],
  ["q6,ε", new Set(["q7"])],
  ["q7,ε", new Set(["q0"])],
]);
console.log("Zustände:", showSet(States));
console.log("Alphabet:", showSet(Sigma));
console.log("Anzahl Übergänge:", delta.size);

const nfa44: NFA = {
  Q: States,
  Sigma,
  delta,
  q0: "q0",
  A: new Set(["q7"]),
};

console.log("NFA erstellt ✅");

Zustände: { q0, q1, q2, q3, q4, q5, q6, q7 }
Alphabet: { a, b }
Anzahl Übergänge: 8
NFA erstellt ✅


States = {'4', '1', '2', '3'}
δ = {
    ('4', 'a'): {'4'},
    ('4', 'b'): {'4','1'},
    ('1', 'a'): {'2'},
    ('1', 'b'): {'2'},
    ('2', 'a'): {'3'},
    ('2', 'b'): {'3'}
}

The non-deterministic <span style="font-variant:small-caps;">Fsm</span> defined below is taken from the lecture notes.

The function `nfa2dot`can be used to render this <span style="font-variant:small-caps;">Fsm</span>.

In [6]:
function nfaToDot(nfa: NFA): string {
  const { delta, q0, A } = nfa;
  const lines: string[] = [];
  lines.push("digraph NFA {");
  lines.push("  rankdir=LR;");

  // Startpfeil
  lines.push('  __start [shape=point, label=""];');
  lines.push(`  __start -> "${q0}";`);

  // akzeptierende Zustände
  lines.push("  node [shape = doublecircle];");
  for (const f of A) lines.push(`  "${f}";`);

  // normale Zustände + Übergänge
  lines.push("  node [shape = circle];");
  for (const [key, targets] of delta.entries()) {
    // ⬇️ Wichtig: am LETZTEN Komma splitten (nicht Zeichen für Zeichen)
    const [from, c] = key.split(/,(?=[^,]*$)/);
    for (const to of targets) {
      lines.push(`  "${from}" -> "${to}" [label="${c}"];`);
    }
  }

  lines.push("}");
  return lines.join("\n");
}

// ======================================================
// 4️⃣ Anzeige im Notebook (Viz.js 3.x)
// ======================================================
async function showNFA(nfa: NFA) {
  const viz = await instance();
  const dot = nfaToDot(nfa);
  const svg = viz.renderString(dot, { format: "svg" });
  display.html(svg);
}

// ======================================================
// 5️⃣ Jetzt anzeigen 🎉
// ======================================================

await showNFA(nfa44);

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.0.1 (20251006.0113)
 -->
<!-- Title: NFA Pages: 1 -->
 
 
 NFA 
 
<!-- __start -->
 
 __start 
 
 
<!-- q0 -->
 
 q0 
 
 q0 
 
<!-- __start->q0 -->
 
 __start->q0 
 
 
 
<!-- q1 -->
 
 q1 
 
 q1 
 
<!-- q0->q1 -->
 
 q0->q1 
 
 
 ε 
 
<!-- q2 -->
 
 q2 
 
 q2 
 
<!-- q0->q2 -->
 
 q0->q2 
 
 
 ε 
 
<!-- q7 -->
 
 q7 
 
 
 q7 
 
<!-- q7->q0 -->
 
 q7->q0 
 
 
 ε 
 
<!-- q3 -->
 
 q3 
 
 q3 
 
<!-- q1->q3 -->
 
 q1->q3 
 
 
 b 
 
<!-- q4 -->
 
 q4 
 
 q4 
 
<!-- q2->q4 -->
 
 q2->q4 
 
 
 a 
 
<!-- q5 -->
 
 q5 
 
 q5 
 
<!-- q3->q5 -->
 
 q3->q5 
 
 
 a 
 
<!-- q6 -->
 
 q6 
 
 q6 
 
<!-- q4->q6 -->
 
 q4->q6 
 
 
 b 
 
<!-- q5->q7 -->
 
 q5->q7 
 
 
 ε 
 
<!-- q6->q7 -->
 
 q6->q7 
 
 
 ε

In [7]:
console.log("Starte NFA → DFA Konvertierung...");
const dfa44: DFA = nfa2dfa(nfa44);
console.log("✅ DFA erfolgreich erstellt");
console.log("Anzahl DFA-Zustände:", dfa44.Q.size);
console.log("Anzahl Akzeptierender Zustände:", dfa44.A.size);

Starte NFA → DFA Konvertierung...
✅ DFA erfolgreich erstellt
Anzahl DFA-Zustände: 6
Anzahl Akzeptierender Zustände: 2


In [8]:
console.log("DFA-Zustände:");

let i = 0;
for (const state of dfa44.Q) {
  console.log(`S${i++} = ${showSet(state)}`);
}

DFA-Zustände:
S0 = { q0, q1, q2 }
S1 = { q4 }
S2 = { q3 }
S3 = {  }
S4 = { q6, q7, q0, q1, q2 }
S5 = { q5, q7, q0, q1, q2 }


This recognizes the same language as the language described by
$$ (a \cdot b + b \cdot a) \cdot (a \cdot b + b \cdot a)^* $$
Let us convert it into a deterministic <span style="font-variant:small-caps;">Fsm</span>: 

In [13]:
import { instance } from "@viz-js/viz";
import { nfa2dfa } from "./01-NFA-2-DFA.js";

// ---- Typen (falls du sie nicht schon importierst) ----
type State = string | number;
type Char = string;
type DFA = {
  Q: Set<Set<State>>;
  Sigma: Set<Char>;
  // Wir nehmen an: delta-Key ist `${JSON.stringify([...M])},${c}`
  delta: Map<string, Set<State>>;
  q0: Set<State>;
  A: Set<Set<State>>;
};

// ---- Hilfsfunktionen für stabile Keys/Parsing ----
function canonSet(s: Set<State>): string {
  return Array.from(s, x => String(x)).sort().join(","); // "" für ∅
}

function labelOfSet(s: Set<State>): string {
  const k = canonSet(s);
  return k === "" ? "∅" : k;
}

// fromKey kommt als '["q0","q1","q2"]' ODER evtl. schon 'q0,q1,q2' oder '' (∅)
function parseFromKey(fromKey: string): Set<State> {
  const trimmed = fromKey.trim();
  if (trimmed === "") return new Set<State>(); // ∅
  if (trimmed.startsWith("[")) {
    try {
      const arr = JSON.parse(trimmed) as (string | number)[];
      return new Set<State>(arr as State[]);
    } catch {
      // Fallback: kommasepariert
    }
  }
  // Fallback: kommaseparierte Liste
  return new Set<State>(trimmed.split(",").filter(s => s.length > 0));
}

// ---- Benennung S0, S1, ... (Startzustand zuerst) ----
function buildNameMap(dfa: DFA): Map<string, string> {
  const allKeys: string[] = [];

  // 1) Startzustand zuerst
  const startKey = canonSet(dfa.q0);
  allKeys.push(startKey);

  // 2) Alle Q-Zustände
  for (const s of dfa.Q) {
    const k = canonSet(s);
    if (!allKeys.includes(k)) allKeys.push(k);
  }

  // 3) Alle from-/to-Zustände aus delta
  for (const [k, tgt] of dfa.delta.entries()) {
    const [fromKeyStr] = k.split(/,(?=[^,]*$)/);
    const fromSet = parseFromKey(fromKeyStr.replace(/[\s]/g, ""));
    const fromK = canonSet(fromSet);
    if (!allKeys.includes(fromK)) allKeys.push(fromK);

    const toK = canonSet(tgt);
    if (!allKeys.includes(toK)) allKeys.push(toK);
  }

  // 4) Reihenfolge stabil halten und Namen vergeben
  const nameMap = new Map<string, string>();
  allKeys.forEach((k, i) => nameMap.set(k, `S${i}`));
  return nameMap;
}

// ---- DOT-Erzeugung mit stabilen Namen ----
function dfaToDotNamed(dfa: DFA): string {
  const names = buildNameMap(dfa);

  const lines: string[] = [];
  lines.push("digraph DFA {");
  lines.push("  rankdir=LR;");

  // Startpfeil
  const startName = names.get(canonSet(dfa.q0)) || "S?";
  lines.push('  __start [shape=point, label=""];');
  lines.push(`  __start -> "${startName}";`);

  // akzeptierende Zustände
  lines.push("  node [shape = doublecircle];");
  for (const F of dfa.A) {
    const name = names.get(canonSet(F)) || "S?";
    lines.push(`  "${name}";`);
  }

  // normale Zustände
  lines.push("  node [shape = circle];");

  // Kanten
  for (const [key, target] of dfa.delta.entries()) {
    const [fromKeyStr, c] = key.split(/,(?=[^,]*$)/);
    const fromSet = parseFromKey(fromKeyStr);
    const fromName = names.get(canonSet(fromSet)) || "S?";

    const toName = names.get(canonSet(target)) || "S?";
    lines.push(`  "${fromName}" -> "${toName}" [label="${c}"];`);
  }

  lines.push("}");
  return lines.join("\n");
}

// ---- Anzeigen im Notebook ----
async function showDFA(dfa: DFA) {
  const viz = await instance();
  const dot = dfaToDotNamed(dfa);
  const svg = viz.renderString(dot, { format: "svg" });
  display.html(svg);
}

// ---- Beispiel: rechne NFA → DFA und zeige ihn ----
const dfa = nfa2dfa(nfa44) as DFA;
await showDFA(dfa);


<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.0.1 (20251006.0113)
 -->
<!-- Title: DFA Pages: 1 -->
 
 
 DFA 
 
<!-- __start -->
 
 __start 
 
 
<!-- S0 -->
 
 S0 
 
 S0 
 
<!-- __start->S0 -->
 
 __start->S0 
 
 
 
<!-- S1 -->
 
 S1 
 
 S1 
 
<!-- S0->S1 -->
 
 S0->S1 
 
 
 a 
 
<!-- S2 -->
 
 S2 
 
 S2 
 
<!-- S0->S2 -->
 
 S0->S2 
 
 
 b 
 
<!-- S4 -->
 
 S4 
 
 
 S4 
 
<!-- S4->S1 -->
 
 S4->S1 
 
 
 a 
 
<!-- S4->S2 -->
 
 S4->S2 
 
 
 b 
 
<!-- S5 -->
 
 S5 
 
 
 S5 
 
<!-- S5->S1 -->
 
 S5->S1 
 
 
 a 
 
<!-- S5->S2 -->
 
 S5->S2 
 
 
 b 
 
<!-- S1->S4 -->
 
 S1->S4 
 
 
 b 
 
<!-- S3 -->
 
 S3 
 
 S3 
 
<!-- S1->S3 -->
 
 S1->S3 
 
 
 a 
 
<!-- S2->S5 -->
 
 S2->S5 
 
 
 a 
 
<!-- S2->S3 -->
 
 S2->S3 
 
 
 b 
 
<!-- S3->S3 -->
 
 S3->S3 
 
 
 a 
 
<!-- S3->S3 -->
 
 S3->S3 
 
 
 b

The function `dfa2dot`can be used to render this <span style="font-variant:small-caps;">Fsm</span>.

In order to inspect the states of this deterministic <span style="font-variant:small-caps;">Fsm</span> we print the dictionary  `states2Names`.

In [20]:
type State = string | number;

interface DFA {
  Q: Set<Set<State>>;
  Sigma: Set<string>;
  delta: Map<string, Set<State>>;
  q0: Set<State>;
  A: Set<Set<State>>;
}

// --- Hilfsfunktionen für stabile Namen ---
function canonSet(s: Set<State>): string {
  return Array.from(s, x => String(x)).sort().join(","); // "" = ∅
}

function printStates2Names(names: Map<string, string>) {
  const entries: string[] = [];
  for (const [k, v] of names.entries()) {
    const pretty =
      k === ""
        ? "frozenset()"
        : `frozenset({${k.split(",").map(s => `'${s}'`).join(", ")}})`;
    entries.push(`${pretty}: '${v}'`);
  }
  console.log("{ " + entries.join(",\n  ") + " }");
}

// --- Mapping: Zustände → S0, S1, ... (Startzustand zuerst) ---
function buildStates2Names(dfa: DFA): Map<string, string> {
  const names = new Map<string, string>();
  const order: string[] = [];

  // Startzustand zuerst
  const startKey = canonSet(dfa.q0);
  order.push(startKey);

  // Alle weiteren Zustände
  for (const S of dfa.Q) {
    const k = canonSet(S);
    if (!order.includes(k)) order.push(k);
  }

  // Zustände aus delta ergänzen (z. B. ∅)
  for (const [, tgt] of dfa.delta.entries()) {
    const tk = canonSet(tgt);
    if (!order.includes(tk)) order.push(tk);
  }

  order.forEach((k, i) => names.set(k, `S${i}`));
  return names;
}

// --- DOT-Erzeugung mit diesen Namen ---
function dfaToDotWithNames(dfa: DFA, names: Map<string, string>): string {
  const lines: string[] = [];
  lines.push("digraph DFA {");
  lines.push("  rankdir=LR;");

  // Startpfeil
  const startName = names.get(canonSet(dfa.q0)) || "S?";
  lines.push('  __start [shape=point, label=""];');
  lines.push(`  __start -> "${startName}";`);

  // akzeptierende Zustände
  lines.push("  node [shape = doublecircle];");
  for (const F of dfa.A) {
    const n = names.get(canonSet(F)) || "S?";
    lines.push(`  "${n}";`);
  }

  // normale Zustände
  lines.push("  node [shape = circle];");

  // Kanten
  for (const [k, tgt] of dfa.delta.entries()) {
    const [fromKeyStr, c] = k.split(/,(?=[^,]*$)/);
    let fromSet: Set<State>;
    try {
      const arr = JSON.parse(fromKeyStr) as (string | number)[];
      fromSet = new Set(arr);
    } catch {
      fromSet = new Set(fromKeyStr ? fromKeyStr.split(",") : []);
    }

    const fromName = names.get(canonSet(fromSet)) || "S?";
    const toName = names.get(canonSet(tgt)) || "S?";
    lines.push(`  "${fromName}" -> "${toName}" [label="${c}"];`);
  }

  lines.push("}");
  return lines.join("\n");
}
async function states2Names(dfa: DFA) {
  const names = buildStates2Names(dfa);
  console.log("states2Names:");
  printStates2Names(names);
}

const dfa = nfa2dfa(nfa44) as DFA;
await states2Names(dfa);

states2Names:
{ frozenset({'q0', 'q1', 'q2'}): 'S0',
  frozenset({'q4'}): 'S1',
  frozenset({'q3'}): 'S2',
  frozenset(): 'S3',
  frozenset({'q0', 'q1', 'q2', 'q6', 'q7'}): 'S4',
  frozenset({'q0', 'q1', 'q2', 'q5', 'q7'}): 'S5' }


We can also print the <span style="font-variant:small-caps;">Fsm</span>.

In [21]:
type State = string | number;

function canonSet(s: Set<State>): string {
  return Array.from(s, x => String(x)).sort().join(","); // "" für ∅
}

function frozensetStr(s: Set<State>): string {
  const elems = Array.from(s).map(x => `'${x}'`).join(", ");
  return elems ? `frozenset({${elems}})` : "frozenset()";
}

/** Gibt ein DFA in Python-ähnlicher Schreibweise aus */
function printFSM(dfa: any) {
  // --- Zustände ---
  const Qs = Array.from(dfa.Q)
    .map((s: Set<State>) => frozensetStr(s))
    .join(",\n  ");
  console.log("Zustände (Q) = {");
  console.log("  " + Qs);
  console.log("}");

  // --- Alphabet ---
  const Sigmas = Array.from(dfa.Sigma)
    .map((c: string) => `'${c}'`)
    .join(", ");
  console.log("\nAlphabet (Σ) = { " + Sigmas + " }");

  // --- Übergänge ---
  console.log("\nÜbergänge (δ) = {");
  for (const [key, target] of dfa.delta.entries()) {
    const [fromKeyStr, c] = key.split(/,(?=[^,]*$)/);
    let fromSet: Set<State>;
    try {
      const arr = JSON.parse(fromKeyStr) as (string | number)[];
      fromSet = new Set(arr);
    } catch {
      fromSet = new Set(fromKeyStr ? fromKeyStr.split(",") : []);
    }
    const left = `(${frozensetStr(fromSet)}, '${c}')`;
    const right = frozensetStr(target);
    console.log(`  ${left}: ${right},`);
  }
  console.log("}");

  // --- Startzustand ---
  console.log("\nStartzustand (q₀) = " + frozensetStr(dfa.q0));

  // --- Akzeptierende Zustände ---
  const finals = Array.from(dfa.A)
    .map((s: Set<State>) => frozensetStr(s))
    .join(",\n  ");
  console.log("\nAkzeptierende Zustände (A) = {");
  console.log("  " + finals);
  console.log("}");
}
const dfa = nfa2dfa(nfa44);
printFSM(dfa);

Zustände (Q) = {
  frozenset({'q0', 'q1', 'q2'}),
  frozenset({'q4'}),
  frozenset({'q3'}),
  frozenset(),
  frozenset({'q6', 'q7', 'q0', 'q1', 'q2'}),
  frozenset({'q5', 'q7', 'q0', 'q1', 'q2'})
}

Alphabet (Σ) = { 'a', 'b' }

Übergänge (δ) = {
  (frozenset({'q0', 'q1', 'q2'}), 'a'): frozenset({'q4'}),
  (frozenset({'q0', 'q1', 'q2'}), 'b'): frozenset({'q3'}),
  (frozenset({'q4'}), 'a'): frozenset(),
  (frozenset({'q4'}), 'b'): frozenset({'q6', 'q7', 'q0', 'q1', 'q2'}),
  (frozenset({'q3'}), 'a'): frozenset({'q5', 'q7', 'q0', 'q1', 'q2'}),
  (frozenset({'q3'}), 'b'): frozenset(),
  (frozenset(), 'a'): frozenset(),
  (frozenset(), 'b'): frozenset(),
  (frozenset({'q0', 'q1', 'q2', 'q6', 'q7'}), 'a'): frozenset({'q4'}),
  (frozenset({'q0', 'q1', 'q2', 'q6', 'q7'}), 'b'): frozenset({'q3'}),
  (frozenset({'q0', 'q1', 'q2', 'q5', 'q7'}), 'a'): frozenset({'q4'}),
  (frozenset({'q0', 'q1', 'q2', 'q5', 'q7'}), 'b'): frozenset({'q3'}),
}

Startzustand (q₀) = frozenset({'q0', 'q1', 'q2'})

Akze